In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
import json
import re

In [3]:
sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

In [4]:
import Reader
import ParamsExtractor3
import DataPreprocessor
import DeepLearningAnomalyDetection2

2024-07-13 13:10:35.487126: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-13 13:10:35.538173: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 13:10:36.335227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import LogLine

/home/rocky/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
l = LogLine.LogLine()

In [7]:
log_types = ['cron', 'laurel', 'maillog', 'messages', 'secure', 'user']
file_paths = ['../../../temp_logs/login01.' + logtype + '.log' for logtype in log_types]
fp = file_paths[0]

In [8]:
fp

'../../../temp_logs/login01.cron.log'

In [9]:
i = 0

with open(fp, 'r') as file:
    for line in file:
        i += 1
        print(line)
        if i>0:
            break

2024-03-27T00:00:01+01:00	{"host":"login01","ident":"CROND","pid":"53766","message":"(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)","severity":"info","facility":"cron","time":"2024-03-27 00:00:01 +0100"}



In [10]:
line

'2024-03-27T00:00:01+01:00\t{"host":"login01","ident":"CROND","pid":"53766","message":"(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)","severity":"info","facility":"cron","time":"2024-03-27 00:00:01 +0100"}\n'

In [11]:
l.print_attributes()

type: None
raw: None
msg: None
severity: None
facility: None
time: None
common_fp_length: None
perimeter_violation: None
temporary_log_key: None
log_key: None
spell_log_key: None
embedding: None


In [12]:
attrs = l.readline(line, 'cron')

In [13]:
l.print_attributes()

type: cron
raw: 2024-03-27T00:00:01+01:00	{"host":"login01","ident":"CROND","pid":"53766","message":"(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)","severity":"info","facility":"cron","time":"2024-03-27 00:00:01 +0100"}

msg: /cinecalocal/scripts/clean_shm_files.sh
severity: 6
facility: 9
time: 2024-03-27 00:00:01 +0100
common_fp_length: -1
perimeter_violation: 0
temporary_log_key:  cinecalocal scripts clean shm files sh
log_key: 101
spell_log_key: None
embedding: [[-5.64066470e-01 -1.27989545e-01 -2.96097994e-01  6.91732317e-02
  -7.34953955e-02 -4.86518025e-01 -2.89425433e-01  2.78283268e-01
  -4.66446847e-01 -3.67658526e-01 -7.99900532e-01  1.39664143e-01
  -2.78952092e-01 -3.57268527e-02  2.81808883e-01  2.43544817e-01
  -2.05919310e-01  6.67244732e-01 -5.16141355e-02 -1.13587409e-01
   6.20730035e-03 -4.99379367e-01 -3.99583459e-01 -6.42248511e-01
  -1.41329274e-01  2.61498149e-02 -7.55563155e-02  2.42415927e-02
  -6.89070106e-01  1.40638892e-02 -4.01434213e-01  1.27061725e

In [14]:
print(attrs)

{'type': 'cron', 'raw': '2024-03-27T00:00:01+01:00\t{"host":"login01","ident":"CROND","pid":"53766","message":"(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)","severity":"info","facility":"cron","time":"2024-03-27 00:00:01 +0100"}\n', 'msg': '/cinecalocal/scripts/clean_shm_files.sh', 'severity': 6, 'facility': 9, 'time': '2024-03-27 00:00:01 +0100', 'common_fp_length': -1, 'perimeter_violation': 0, 'temporary_log_key': ' cinecalocal scripts clean shm files sh', 'log_key': 101, 'spell_log_key': None, 'embedding': array([[-5.64066470e-01, -1.27989545e-01, -2.96097994e-01,
         6.91732317e-02, -7.34953955e-02, -4.86518025e-01,
        -2.89425433e-01,  2.78283268e-01, -4.66446847e-01,
        -3.67658526e-01, -7.99900532e-01,  1.39664143e-01,
        -2.78952092e-01, -3.57268527e-02,  2.81808883e-01,
         2.43544817e-01, -2.05919310e-01,  6.67244732e-01,
        -5.16141355e-02, -1.13587409e-01,  6.20730035e-03,
        -4.99379367e-01, -3.99583459e-01, -6.42248511e-01,
    

In [33]:
attrs['embedding'].shape

(1, 768)

In [15]:
data_list = []

index = line.find('{')
if index != -1:
    json_data = line[index:]
    data = json.loads(json_data)
    data_list.append(data)

In [16]:
data['message']

'(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)'

In [17]:
data_list

[{'host': 'login01',
  'ident': 'CROND',
  'pid': '53766',
  'message': '(root) CMD (/cinecalocal/scripts/clean_shm_files.sh)',
  'severity': 'info',
  'facility': 'cron',
  'time': '2024-03-27 00:00:01 +0100'}]

In [18]:
df = pd.DataFrame(data_list)
df

,host,ident,pid,message,severity,facility,time
0,login01,CROND,53766,(root) CMD (/cinecalocal/scripts/clean_shm_fil...,info,cron,2024-03-27 00:00:01 +0100


In [19]:
def remove_numbers_and_special_characters(input_string):
    # Define the regex pattern to match any character that is not a letter
    pattern = r'[^a-zA-Z\s]'
    # Replace all matches with an empty string
    cleaned_string = re.sub(pattern, ' ', input_string)
    return cleaned_string

In [20]:
remove_numbers_and_special_characters(data['message'])

' root  CMD   cinecalocal scripts clean shm files sh '

In [21]:
i = 0
data_list = []

with open(fp, 'r') as file:
    for line in file:
        index = line.find('{')
        if index != -1:
            json_data = line[index:]
            data = json.loads(json_data)
            data_list.append(remove_numbers_and_special_characters(data['message']))
        i += 1
        if i>10:
            break

In [22]:
data_list

[' root  CMD   cinecalocal scripts clean shm files sh ',
 ' root  CMD    cinecalocal nagios passive passive pl   delay      cronid      dev null      ',
 ' a  cmc    CMD  source  g    work CMCC medfs   download set env sh  python  g    work CMCC medfs   download dasDwlSystem mp dwld driver py    g    work CMCC medfs   download LOGS dasDwlSystem mp dwld driver log  date   Y m d H M S  log      ',
 ' a  cmc    CMD   g    work CMCC medfs   download dasDwlRawScripts ecmwfRollingRm sh     g    work CMCC medfs   download LOGS ecmwfRolling log      ',
 ' root  CMD  run parts  etc cron hourly ',
 '  etc cron hourly  starting  anacron',
 'Anacron started on           ',
 'Normal exit    jobs run ',
 '  etc cron hourly  finished  anacron',
 ' root  CMD   bin logger  STATUS CHECK   p local  info     tmp lastcron remote log status ',
 ' root  CMD    cinecalocal nagios passive passive pl   delay      cronid      dev null      ']

In [23]:
def reduce_double_spaces(input_string):
    # Define the regex pattern to match two or more spaces
    pattern = r'\s{2,}'
    # Replace all matches with a single space
    reduced_string = re.sub(pattern, ' ', input_string)
    return reduced_string

In [24]:
for i in range(len(data_list)):
    data_list[i] = reduce_double_spaces(data_list[i])

In [25]:
data_list

[' root CMD cinecalocal scripts clean shm files sh ',
 ' root CMD cinecalocal nagios passive passive pl delay cronid dev null ',
 ' a cmc CMD source g work CMCC medfs download set env sh python g work CMCC medfs download dasDwlSystem mp dwld driver py g work CMCC medfs download LOGS dasDwlSystem mp dwld driver log date Y m d H M S log ',
 ' a cmc CMD g work CMCC medfs download dasDwlRawScripts ecmwfRollingRm sh g work CMCC medfs download LOGS ecmwfRolling log ',
 ' root CMD run parts etc cron hourly ',
 ' etc cron hourly starting anacron',
 'Anacron started on ',
 'Normal exit jobs run ',
 ' etc cron hourly finished anacron',
 ' root CMD bin logger STATUS CHECK p local info tmp lastcron remote log status ',
 ' root CMD cinecalocal nagios passive passive pl delay cronid dev null ']

In [26]:
import nltk
from nltk.tokenize import word_tokenize
# Download the necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/rocky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
def nltk_word_tokenizer(input_string):
    # Use nltk's word_tokenize to split the string into tokens
    tokens = word_tokenize(input_string)
    return tokens

In [28]:
token_list = []
for d in data_list:
    tokens = nltk_word_tokenizer(d)
    token_list.append(tokens)
    #print(tokens)
token_list

[['root', 'CMD', 'cinecalocal', 'scripts', 'clean', 'shm', 'files', 'sh'],
 ['root',
  'CMD',
  'cinecalocal',
  'nagios',
  'passive',
  'passive',
  'pl',
  'delay',
  'cronid',
  'dev',
  'null'],
 ['a',
  'cmc',
  'CMD',
  'source',
  'g',
  'work',
  'CMCC',
  'medfs',
  'download',
  'set',
  'env',
  'sh',
  'python',
  'g',
  'work',
  'CMCC',
  'medfs',
  'download',
  'dasDwlSystem',
  'mp',
  'dwld',
  'driver',
  'py',
  'g',
  'work',
  'CMCC',
  'medfs',
  'download',
  'LOGS',
  'dasDwlSystem',
  'mp',
  'dwld',
  'driver',
  'log',
  'date',
  'Y',
  'm',
  'd',
  'H',
  'M',
  'S',
  'log'],
 ['a',
  'cmc',
  'CMD',
  'g',
  'work',
  'CMCC',
  'medfs',
  'download',
  'dasDwlRawScripts',
  'ecmwfRollingRm',
  'sh',
  'g',
  'work',
  'CMCC',
  'medfs',
  'download',
  'LOGS',
  'ecmwfRolling',
  'log'],
 ['root', 'CMD', 'run', 'parts', 'etc', 'cron', 'hourly'],
 ['etc', 'cron', 'hourly', 'starting', 'anacron'],
 ['Anacron', 'started', 'on'],
 ['Normal', 'exit', 'jobs'

In [29]:
def text_to_bow(input_string):
    # Tokenize the input string
    tokens = nltk_word_tokenizer(input_string)
    # Join tokens to form the processed string
    processed_string = ' '.join(tokens)
    
    # Initialize the CountVectorizer
    vectorizer = CountVectorizer()
    # Fit and transform the processed string
    bow_representation = vectorizer.fit_transform([processed_string])
    
    return bow_representation.toarray(), vectorizer.get_feature_names_out()

In [30]:
# Example usage
from sklearn.feature_extraction.text import CountVectorizer

bow_representation, feature_names = text_to_bow(data_list[0])
print("Bag of Words Representation:\n", bow_representation)
print("Feature Names:\n", feature_names)

Bag of Words Representation:
 [[1 1 1 1 1 1 1 1]]
Feature Names:
 ['cinecalocal' 'clean' 'cmd' 'files' 'root' 'scripts' 'sh' 'shm']


In [31]:
def text_to_tfidf(input_string):
    # Tokenize the input string
    tokens = nltk_word_tokenizer(input_string)
    # Join tokens to form the processed string
    processed_string = ' '.join(tokens)
    
    # Initialize the TfidfVectorizer
    vectorizer = TfidfVectorizer()
    # Fit and transform the processed string
    tfidf_representation = vectorizer.fit_transform([processed_string])
    
    return tfidf_representation.toarray(), vectorizer.get_feature_names_out()

In [32]:
# Example usage
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_representation, feature_names = text_to_tfidf(data_list[0])
print("TF-IDF Representation:\n", tfidf_representation)
print("Feature Names:\n", feature_names)

TF-IDF Representation:
 [[0.35355339 0.35355339 0.35355339 0.35355339 0.35355339 0.35355339
  0.35355339 0.35355339]]
Feature Names:
 ['cinecalocal' 'clean' 'cmd' 'files' 'root' 'scripts' 'sh' 'shm']
